Read Data

In [1]:
import pandas as pd

offers = pd.read_csv(r'C:\\Users\\chcww\\Downloads\\offers.csv')
transactions = pd.read_csv(r'C:\\Users\\chcww\\Downloads\\newdata.csv')
trainHistory = pd.read_csv(r'C:\\Users\\chcww\\Downloads\\trainHistory.csv')
testHistory = pd.read_csv(r'C:\\Users\\chcww\\Downloads\\testHistory.csv')

In [1]:
import pandas as pd

offertry = pd.read_csv(r'C:\\Users\\chcww\\Downloads\\offers.csv')

In [2]:
offertry.columns

Index(['offer', 'category', 'quantity', 'company', 'offervalue', 'brand'], dtype='object')

In [4]:

offertry.rename(columns = {'offer' : '12', 'category' : '132', 'quantity' : '1232', 
                           'company' : '125232', 'offervalue' : '12352', 'brand' : '12332'}, inplace = True)

Merge Data

In [4]:
train = pd.merge(trainHistory, offers, on='offer')
train = pd.merge(train, transactions, on='id')[:10000,]

test = pd.merge(testHistory, offers, on='offer')
test = pd.merge(test, transactions, on='id')[:10000,]

data = pd.concat([train, test], axis=0, ignore_index=True)

InvalidIndexError: (slice(None, 10000, None),)

Offer One-Hot

In [6]:
trainOffer = trainHistory[['offer', 'id']]
trainOneHot = pd.get_dummies(trainOffer, columns = ['offer'])

In [11]:
# Extract features
features = ['chain', 'offer', 'market', 'category', 'company', 'brand', 'productsize', 'productmeasure', 'purchasequantity', 'purchaseamount']

# Feature engineering
# 1. Group the data by customer and offer, and calculate aggregate features
grouped = data.groupby(['id', 'offer']).agg({
    'market': 'first',
    'productsize': 'first',
    'productmeasure': 'first',
    'quantity': 'sum',
    'offervalue': 'first'
})

# Rename columns
grouped.columns = ['market', 'productsize', 'productmeasure', 'totalquantity', 'offervalue']

# Reset index to make customer and offer into separate columns
grouped = grouped.reset_index()

# 2. Calculate additional features
# a) Average transaction amount for each customer
avg_transaction_amount = data.groupby(['id'])['purchaseamount'].mean().reset_index()
avg_transaction_amount.columns = ['id', 'avg_transaction_amount']
grouped = pd.merge(grouped, avg_transaction_amount, on='id')



#calculate additional features average transaction amount for each customer

In [ ]:

# b) Total amount spent by each customer
total_amount_spent = data.groupby(['id'])['purchaseamount'].sum().reset_index()
total_amount_spent.columns = ['id', 'total_amount_spent']
grouped = pd.merge(grouped, total_amount_spent, on='id')

# c) Number of offers used by each customer
num_offers_used = data.groupby(['id'])['offer'].nunique().reset_index()
num_offers_used.columns = ['id', 'num_offers_used']
grouped = pd.merge(grouped, num_offers_used, on='id')

# d) Total number of transactions by each customer
num_transactions = data.groupby(['id'])['transacted'].count().reset_index()
num_transactions.columns = ['id', 'num_transactions']
grouped = pd.merge(grouped, num_transactions, on='id')

# e) Average quantity purchased by each customer
avg_quantity = data.groupby(['id'])['quantity'].mean().reset_index()
avg_quantity.columns = ['id', 'avg_quantity']
grouped = pd.merge(grouped, avg_quantity, on='id')

# f) Total quantity purchased by each customer
total_quantity = data.groupby(['id'])['quantity'].sum().reset_index()
total_quantity.columns = ['id', 'total_quantity']
grouped = pd.merge(grouped, total_quantity, on='id')

# Split the data back into train and test sets
train_data = grouped[grouped['offer'].isin(trainHistory['offer'])]
test_data = grouped[grouped['offer'].isin(testHistory['offer'])]